In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### 1단계

In [2]:
# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [7]:

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()
print(type(documents[0]),documents[0].page_content[:20])

<class 'langchain_core.documents.base.Document'> 제12조(비과세소득) 다음 각 호의 


In [10]:

# 2️. Text Split
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(documents)
print(len(split_docs),split_docs[11])

12 page_content='아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득
    　　　　1) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따른 종교관련종사자(이하 “종교관련종사자”라 한다)가 받는 대통령령으로 정하는 학자금
    　　　　2) 종교관련종사자가 받는 대통령령으로 정하는 식사 또는 식사대
    　　　　3) 종교관련종사자가 받는 대통령령으로 정하는 실비변상적 성질의 지급액
    　　　　4) 종교관련종사자 또는 그 배우자의 출산이나 6세 이하(해당 과세기간 개시일을 기준으로 판단한다) 자녀의 보육과 관련하여 종교단체로부터 받는 금액으로서 월 20만원 이내의 금액
    　　　　5) 종교관련종사자가 기획재정부령으로 정하는 사택을 제공받아 얻는 이익
    자. 법령ㆍ조례에 따른 위원회 등의 보수를 받지 아니하는 위원(학술원 및 예술원의 회원을 포함한다) 등이 받는 수당' metadata={'source': 'data/taxinfo.txt'}


In [11]:

# 3️. Indexing (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
# 로컬 파일로 저장
vectorstore.save_local("faiss_index")
print(vectorstore)

In [ ]:

# 4️. Retrieval (유사 문서 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
# **질문(쿼리)**에 대해 유사한 문서를 검색하는 역할
retrieved_docs = retriever.invoke("소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print(type(retrieved_docs))
pprint(retrieved_docs)


In [ ]:

# 5️. Generation (LLM 응답 생성)
llm = ChatOpenAI(model="gpt-4o")
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(context)

In [ ]:

response_context = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}")
print('context 적용한 결과')
pprint(response_context.content)


In [ ]:

response = llm.invoke(f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?")
print('context 적용하지 않은 결과')
pprint(response.content)

### 2단계

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. 데이터 로드 (기존과 동일)
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

# 2. 텍스트 분할 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 크기 증가
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],  # 자연스러운 분할을 위한 구분자
    length_function=len,
    is_separator_regex=False,
)
split_docs = splitter.split_documents(documents)

# 3. 인덱싱 (벡터 저장)
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. 검색 개선
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 최대 다양성 검색
    search_kwargs={"k": 5, "fetch_k": 10}  # 더 많은 결과 검색
)

# 5. 프롬프트 엔지니어링
def generate_prompt(query, context):
    return f"""다음은 소득세법 비과세소득 관련 조항입니다. 문맥을 고려하여 질문에 답변하세요.

[관련 조항]
{context}

[질문]
{query}

[답변 요구사항]
- 비과세소득 유형을 계층적으로 구분하여 설명
- 각 항목별 구체적인 조건 명시
- 법조문의 항, 호, 목 번호를 포함
- 500자 이내로 간결하게 요약"""

# 검색 및 응답 생성
query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)  # 창의성 낮춤
response = llm.invoke(generate_prompt(query, context))

print('개선된 결과:')
pprint(response.content)

### 3단계

In [ ]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

# 1. Load Data
loader = TextLoader("data/taxinfo.txt", encoding="utf-8")
documents = loader.load()

print("=== 원본 문서 길이 ===")
print(f"전체 문서 길이: {len(documents[0].page_content)} 글자")

# 2. Text Split 개선
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # 500 → 800 (법령 조항이 길어서)
    chunk_overlap=150,  # 50 → 150 (맥락 보존 강화)
    separators=["\n\n", "\n", ". ", " ", ""]  # 법령 구조에 맞는 분리자
)
split_docs = splitter.split_documents(documents)

print(f"분할된 문서 수: {len(split_docs)}개")
print("=== 분할 예시 ===")
for i, doc in enumerate(split_docs[:3]):
    print(f"Chunk {i+1} ({len(doc.page_content)}글자): {doc.page_content[:100]}...")

# 3. Indexing
vectorstore = FAISS.from_documents(split_docs, OpenAIEmbeddings())
vectorstore.save_local("faiss_index")

# 4. Retrieval 개선
retriever = vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 6}  # 2 → 6으로 증가
)

query = "소득세법에서 비과세소득에 해당하는 소득은 무엇인가요?"
retrieved_docs = retriever.invoke(query)

print(f"\n=== 검색된 문서 ({len(retrieved_docs)}개) ===")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")
    print("---")

# 5. Generation - 개선된 프롬프트
llm = ChatOpenAI(model="gpt-4o", temperature=0)
context = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])

# 개선된 프롬프트 - 더 구체적인 지시사항
improved_prompt = f"""
당신은 세무 전문가입니다. 아래 소득세법 제12조 조항을 바탕으로 질문에 답변해주세요.

질문: {query}

법령 조항:
{context}

다음 형식으로 답변해주세요:
1. 비과세소득의 정의
2. 주요 비과세소득 항목들을 다음과 같이 분류:
   - 사업소득 관련
   - 근로소득/퇴직소득 관련  
   - 연금소득 관련
   - 기타소득 관련
3. 각 항목별 구체적인 조건이나 한도액 명시

답변은 법조문을 인용하면서 구체적으로 작성해주세요.
"""

# 비교용 - 기존 방식
simple_prompt = f"소득세법에서 비과세소득에 해당하는 소득은 무엇인가요? 관련 정보: {context}"

print("\n=== 개선된 프롬프트로 답변 ===")
response_improved = llm.invoke(improved_prompt)
pprint(response_improved.content)

print("\n" + "="*50)
print("=== 기존 프롬프트로 답변 ===")
response_simple = llm.invoke(simple_prompt)
pprint(response_simple.content)

# 추가 개선: 다른 검색 방식 시도
print("\n" + "="*50)
print("=== 검색 방식 개선 테스트 ===")


In [20]:

# MMR(Maximum Marginal Relevance) 검색 - 다양성 확보
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 6, "fetch_k": 20}
)
retrieved_docs_mmr = retriever_mmr.invoke(query)
context_mmr = "\n\n".join([f"[문서 {i+1}]\n{doc.page_content}" for i, doc in enumerate(retrieved_docs_mmr)])

response_mmr = llm.invoke(f"""
{query}

법령 조항 (MMR 검색):
{context_mmr}

위 법령을 바탕으로 비과세소득 항목들을 체계적으로 정리해주세요.
""")

print("=== MMR 검색 결과 ===")
pprint(response_mmr.content)

=== MMR 검색 결과 ===
('소득세법 제12조에 따른 비과세소득 항목들을 다음과 같이 체계적으로 정리할 수 있습니다.\n'
 '\n'
 '1. **공익신탁의 이익**\n'
 '   - 「공익신탁법」에 따른 공익신탁의 이익\n'
 '\n'
 '2. **사업소득**\n'
 '   - 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n'
 '   - 1개의 주택을 소유하는 자의 주택임대소득 (단, 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 '
 '제외)\n'
 '   - 대통령령으로 정하는 농어가부업소득\n'
 '   - 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n'
 '   - 조림기간 5년 이상인 임지의 임목의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액\n'
 '   - 대통령령으로 정하는 작물재배업에서 발생하는 소득\n'
 '   - 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n'
 '\n'
 '3. **근로소득과 퇴직소득**\n'
 '   - 대통령령으로 정하는 복무 중인 병이 받는 급여\n'
 '   - 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n'
 '   - 「산업재해보상보험법」에 따라 수급권자가 받는 각종 보상금\n'
 '   - 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받는 각종 보상금\n'
 '   - 대통령령으로 정하는 복리후생적 성질의 급여\n'
 '\n'
 '4. **연금소득**\n'
 '   - 공적연금 관련법에 따라 받는 각종 유족연금 및 장애연금\n'
 '   - 「산업재해보상보험법」에 따라 받는 각종 연금\n'
 '   - 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금\n'
 '\n'
 '5. **기타소득**\n'
 '   - 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금 및 학습보조비\n'
 '   - 「북한이탈주민의